# Laptop Product Data - Flipkart

In [1]:
import time
import pandas as pd    
# ------------- # 
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import json
from bs4 import BeautifulSoup
import re

In [2]:
driver = webdriver.Chrome()  

# Maximize Window
driver.maximize_window() 
driver.minimize_window()  
driver.maximize_window()  
driver.switch_to.window(driver.current_window_handle)
driver.implicitly_wait(10)

AllProductLinks = []

driver.get('https://www.flipkart.com/search?q=laptop&sid=6bo%2Cb5g&as=on&as-show=on&otracker=AS_QueryStore_HistoryAutoSuggest_4_0_na_na_na&otracker1=AS_QueryStore_HistoryAutoSuggest_4_0_na_na_na&as-pos=4&as-type=HISTORY&suggestionId=laptop%7CLaptops&requestId=652c9576-0092-4a93-b032-bf90ba1121d1')

#========================getting products links to navigate further product pages===================================#
for i in range(3):
    
    products_tag = driver.find_elements(By.CLASS_NAME, '_1fQZEK')

    product_links = [i.get_attribute('href') for i in products_tag]
    
    AllProductLinks.extend(product_links)
    
    driver.find_element(By.XPATH,"//span[contains(text(),'Next')]").click()
    
    time.sleep(3)

print(len(AllProductLinks))
pd.DataFrame(AllProductLinks).to_csv(r'D:\VS_Code\FlipkartElect\AllProductLinks.csv',encoding = 'utf-8-sig',index = False)


0


""


In [4]:
#================================details to be extracted from each product page=============================#
details = {'Name':[],'Rating':[],'CountReviewsratings':[],'DiscountPrice':[],
           'OriginalPrice':[],'DiscountPercent':[],'BankOfffers':[],'DeliveryDetails':[],'Specification':[], 'Links':[]}

In [ ]:
#===========================read product details from the files to navigate========================#
AllLinks = pd.read_csv(r"D:\VS_Code\FlipkartElect\AllProductLinks.csv")
AllProductLinks = AllLinks['0'].to_list()
detailsframe = pd.read_csv(r'D:\VS_Code\FlipkartElect\LaptopDataHarvestMaestro.csv',encoding = 'utf-8-sig')

details = detailsframe.to_dict('list')

wait_time = 0

In [ ]:
#==========================open the product links one by one and extarct the details============================#

for link in AllProductLinks:
    print(link)
    if link not in details['Links']:
        
        print(link)
        op = webdriver.ChromeOptions() #giving headless options to not show the browser
        op.add_argument('headless')
        driver = webdriver.Chrome(options=op)
        #driver = webdriver.Chrome()
        driver.get(link)
        # Maximize Window
        driver.maximize_window() 
        # driver.implicitly_wait(10)
        
        try:driver.find_element(By.CLASS_NAME,'IMZJg1').click()
        except:pass
        
        try:driver.find_element(By.XPATH,"//button[contains(text(),'Read More')]").click()
        except:pass
        
        soup = BeautifulSoup(driver.page_source,'html.parser')
        
        #ProductValues = driver.find_element(By.CLASS_NAME,'aMaAEs')

        ProductValues = soup.find('div','aMaAEs')

        try:Name=ProductValues.find('span','B_NuCI').text
        except:Name='-'

        try:Rating = ProductValues.find('div','_3LWZlK').text
        except:Rating = '-'

        try:CountReviewsratings=ProductValues.find('span','_2_R_DZ').text
        except:CountReviewsratings='-'

        try:DiscountPrice=ProductValues.find('div','_30jeq3').text
        except:DiscountPrice='-'

        try:OriginalPrice=ProductValues.find('div','_3I9_wc').text
        except:OriginalPrice='-'
        
        try:DiscountPercent=ProductValues.find('div','_3Ay6Sb').text
        except:DiscountPercent='-'
        
        try:BankOfffers=[i.text for i in soup.find_all('span','_3j4Zjq')]
        except:BankOfffers='-'
        
        try:DeliveryDetails=soup.find('div','_3XINqE').text
        except:DeliveryDetails='-'
        
        #=================================specifications collection==========================================#
        try:
            SpecTitles = [ SpecTitle.text for SpecTitle in soup.find_all('div','flxcaE') ]

            SpecsFull = {}

            for df,spectitle in zip(pd.read_html(link),SpecTitles):
                
                SpecKeys= df[0].to_list()
                SpecValues = df[1].to_list()
                
                SpecsDict = {}
                
                for i,j in zip(SpecKeys,SpecValues):
                    SpecsDict.update({i:j})
                
                SpecsFull.update({spectitle:SpecsDict})
                
            details['Specification'].append(SpecsFull)
                
        except:
            
            details['Specification'].append('-')
        
        details['Name'].append(Name)
        details['Rating'].append(Rating)
        details['CountReviewsratings'].append(CountReviewsratings)
        details['DiscountPrice'].append(DiscountPrice)
        details['OriginalPrice'].append(OriginalPrice)
        details['DiscountPercent'].append(DiscountPercent)
        details['BankOfffers'].append(BankOfffers)
        details['DeliveryDetails'].append(DeliveryDetails)
        details['Links'].append(link)
        
        driver.close()
        
        print(Name,Rating,CountReviewsratings,DiscountPrice,OriginalPrice,DiscountPercent,BankOfffers,DeliveryDetails)
        wait_time = wait_time + 1
        
        if wait_time%100==0:

            time.sleep(3)
            break 
        
    print(details)

In [ ]:
#=================================Convert into dataframe and save it to csv file=============================#
#details['Links'] = AllProductLinks[:118]
pd.DataFrame(details).to_csv('D:\VS_Code\FlipkartElect\LaptopDataHarvestMaestro.csv',encoding = 'utf-8-sig',index = False)

In [ ]:
print(soup.prettify())